In [ ]:
#============================================================================
# calc_cloud_properties_varying_Zemin_Dmin.py
#
# Calculates the precipitation occurrence frequency according to
# various dmin and ze_thresh detection methods.
#
#
# Author: McKenna W. Stanford

#============================================================================

In [ ]:
#--------------------------------
# Imports
#--------------------------------
import numpy as np
import matplotlib.pyplot as plt
import glob
import xarray
import datetime
import calendar
from matplotlib.gridspec import GridSpec
import matplotlib.dates as mdates
import matplotlib
import pickle
import pandas as pd
import os
from file_struct import file_struct as fs
from load_sonde_data import load_sonde_data
from give_me_files_and_subfolders import give_me_files_and_subfolders
from scipy import ndimage
from scipy.ndimage import gaussian_filter
from scipy.interpolate import NearestNDInterpolator as nn
from matplotlib.patches import Rectangle
from matplotlib import cm
import matplotlib.ticker as ticker
from scipy import stats
from dask.distributed import Client, progress, LocalCluster
import time
import ctypes


In [ ]:
#--------------------------------------------
# Functions 
#--------------------------------------------

#--------------------------------------------
# Occurrence Frequency
#--------------------------------------------
def occur_freq(data,bins,full_length):
    tmp_hist = []
    for ii in range(len(bins)-1):
        if ii != (len(bins)-2):
            tmpid = np.where((data >= bins[ii]) & (data < bins[ii+1]))
            tmp_hist.append(np.size(tmpid)/full_length)
        elif ii == (len(bins)-2):
            tmpid = np.where((data >= bins[ii]) & (data <= bins[ii+1]))
            tmp_hist.append(np.size(tmpid)/full_length)            
    tmp_hist = np.array(tmp_hist)
    return tmp_hist,bins
#--------------------------------------------
# Trim Memory
#--------------------------------------------
def trim_memory() -> int:
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

In [ ]:
#--------------------------------------------
#--------------------------------------------
path = '/mnt/raid/mwstanfo/micre_precip_varying_detections/v8/'
#files = glob.glob(path+'micre_precip_varying_detections_*201*_v8.p')
#files = glob.glob(path+'micre_precip_varying_detections_*201*_v8_lowbias25.p')
files = glob.glob(path+'micre_precip_varying_detections_*201*_v8_lowbias50.p')
files = sorted(files)
#files = files[250:260]
num_files = len(files)
print(num_files)

In [ ]:
cluster = LocalCluster(n_workers=4,threads_per_worker=1,memory_limit='20GB',processes=True)
cluster

In [ ]:
client = Client(cluster)
client

In [ ]:
client.shutdown()

In [ ]:
def read_in_file(infile,vars_dict):
    pkl_dict = pickle.load(open(infile,"rb"))
    dum_key = list(pkl_dict.keys())[0]
    pkl_dict = pkl_dict[dum_key]
    
    keys = ['ctt','cbt','precip_binary','cloud_binary',\
            'precip_binary','bad_radar_data_mask','sonde_sufficient_mask',\
            'precip_to_sfc_binary',\
            'ref_mean_bel',\
            'precip_updraft_binary',\
            'sfc_precip_binary','sfc_precip_updraft_binary',\
            'sfc_ref_mean']
    
    dmin_arr = np.arange(50,650,50)
    ze_thresh_arr = np.array([-55,-50,-45,-40,-35,-30,-25,-20,-15,-10,-5,0,5,10,15])            
    
    for dmin in dmin_arr:
        dmin_str = str(dmin)
        for ze_thresh in ze_thresh_arr:
            ze_thresh_str = str(ze_thresh)
            
            var_dict = {}
            for key in keys:
                    var_dict[key] = pkl_dict[dmin_str][ze_thresh_str][key]
                        
            vars_dict[dmin_str][ze_thresh_str] = var_dict
                        
    return vars_dict

In [ ]:
dmin_arr = np.arange(50,650,50)
ze_thresh_arr = np.array([-55,-50,-45,-30,-35,-30,-25,-20,-15,-10,-5,0,5,10,15])
vars_dict = {}
for dmin in dmin_arr:
    dmin_str = str(dmin)
    vars_dict[dmin_str] = {}
    for ze_thresh in ze_thresh_arr:
        ze_thresh_str = str(ze_thresh)
        vars_dict[dmin_str][ze_thresh_str] = None

In [ ]:
futures_dict = []
for file in files:
    #print(file)
    future_dict = client.submit(read_in_file,file,vars_dict)
    #future_dict = read_in_file(file,vars_dict)
    futures_dict.append(future_dict)
results_dict = client.gather(futures_dict)
print('done')

In [ ]:
def calc_precip_occur_freq(dmin,ze_thresh,results_dict):

        dmin_str = str(dmin)
        ze_thresh_str = str(ze_thresh)
        #print('Dmin:',dmin_str)
        #print('Ze_thresh:',ze_thresh_str)
                
        keys = ['ctt','cbt','precip_binary','cloud_binary',\
                'precip_binary','bad_radar_data_mask','sonde_sufficient_mask',\
                'precip_to_sfc_binary',\
                'ref_mean_bel',\
                'precip_updraft_binary',\
                'sfc_precip_binary','sfc_precip_updraft_binary',\
                'sfc_ref_mean']
        
        var_dict_2 = {}
        
        
        for key in keys:
            var_dict_2[key] = []        
        
        
        for kk in range(len(results_dict)):
            out_var_dict = results_dict[kk][dmin_str][ze_thresh_str]
            for key in out_var_dict.keys():
                var_dict_2[key].append(out_var_dict[key])
                    
        #--------------------------------------------------
        #--------------------------------------------------
        #--------------------------------------------------
        # Concatenate Dates
        #--------------------------------------------------
        #--------------------------------------------------
        #--------------------------------------------------
        for key in var_dict_2.keys():
            #print(key)
            var_dict_2[key] = np.concatenate(var_dict_2[key])
        
        dumid = np.where( (var_dict_2['sonde_sufficient_mask'] > 0.) & (var_dict_2['bad_radar_data_mask'] == 0.) )
        dumsize1 = np.size(var_dict_2['sonde_sufficient_mask'])
        dumsize2 = np.size(dumid)
        #print('Processing:')
        #print('  Total # of data points: {}'.format(dumsize1))
        #print('  Total # of data points after removing insufficient sonde mask and bad radar data mask: {}'.format(dumsize2))
        #print('  Total # of data points after filtering: {} %'.format(dumsize2/dumsize1*100.))       

        #---------------------------------------------------------------
        #---------------------------------------------------------------
        #---------------------------------------------------------------
        # Limit all variable arrays to only include values with sufficient sonde data and good radar data
        #---------------------------------------------------------------
        #---------------------------------------------------------------
        #---------------------------------------------------------------        
        
        for key in var_dict_2.keys():
            #print(key)
            var_dict_2[key] = var_dict_2[key][dumid]     
            
        #-------------------------------------------------------------------
        #-------------------------------------------------------------------
        #-------------------------------------------------------------------
        # Create new dictionary that is limited to only points with cloud
        #-------------------------------------------------------------------
        #-------------------------------------------------------------------
        #-------------------------------------------------------------------
        cloud_id = np.where(var_dict_2['cloud_binary'] == 1.)[0]
        var_dict_cloud = var_dict_2.copy()
        for key in var_dict_cloud.keys():
            #print(key)
            var_dict_cloud[key] = var_dict_cloud[key][cloud_id]
        #return var_dict_cloud
        
        #return var_dict_cloud
        ## Limit to CBT and CTT < -38 deg C
        dumid = np.where( (var_dict_cloud['cbt'] > -38.) & (var_dict_cloud['ctt'] > -38.))
        for key in var_dict_cloud.keys():
            var_dict_cloud[key] = var_dict_cloud[key][dumid]
        #print(np.size(dumid))
        #print(np.min(var_dict_2['cbt'][dumid]))
        #print(np.min(var_dict_2['ctt'][dumid]))
        #print(aaaa)
    
        #-------------------------------------------------------------------
        #-------------------------------------------------------------------
        #-------------------------------------------------------------------
        # Remove updrafts
        #-------------------------------------------------------------------
        #-------------------------------------------------------------------
        #-------------------------------------------------------------------
        no_updraft_id = np.where((var_dict_cloud['precip_updraft_binary'] == 0.) | (var_dict_cloud['precip_updraft_binary'] == -999.))[0]
        for key in var_dict_cloud.keys():
            #print(key)
            var_dict_cloud[key] = var_dict_cloud[key][no_updraft_id]        

            
            
        #-------------------------------------------------------------------
        #-------------------------------------------------------------------
        #-------------------------------------------------------------------
        # Modify precipitation occurrence frequency
        #-------------------------------------------------------------------
        #-------------------------------------------------------------------
        #-------------------------------------------------------------------
        tmp_precip_binary = var_dict_cloud['precip_binary']
        tmp_sfc_precip_binary = var_dict_cloud['sfc_precip_binary']
        tmp_ref_mean_bel = var_dict_cloud['ref_mean_bel']
        tmp_sfc_ref_mean = var_dict_cloud['sfc_ref_mean']
        tmp_precip_binary_2 = tmp_precip_binary.copy()
        tmp_sfc_precip_binary_2 = tmp_sfc_precip_binary.copy()
        
        ref_id = np.where(tmp_ref_mean_bel >= ze_thresh)
        tmp_precip_binary_2[ref_id] = 1.
        no_ref_id = np.where((tmp_ref_mean_bel < ze_thresh) & (tmp_ref_mean_bel > -999.))
        tmp_precip_binary_2[no_ref_id] = 0.
        
        sfc_ref_id = np.where(tmp_sfc_ref_mean >= ze_thresh)
        tmp_sfc_precip_binary_2[sfc_ref_id] = 1.
        no_sfc_ref_id = np.where((tmp_sfc_ref_mean < ze_thresh) & (tmp_sfc_ref_mean > -999.))
        tmp_sfc_precip_binary_2[no_sfc_ref_id] = 0.
        
        var_dict_cloud['precip_binary'] = tmp_precip_binary_2
        var_dict_cloud['sfc_precip_binary'] = tmp_sfc_precip_binary_2
        
            
            
        #-------------------------------------------------------------------
        #-------------------------------------------------------------------
        #-------------------------------------------------------------------
        # Now calculate precipitation occurrence frequency
        #-------------------------------------------------------------------
        #-------------------------------------------------------------------
        #-------------------------------------------------------------------
        
        # Elevated clouds
        cloud_id = np.where((var_dict_cloud['cloud_binary'] == 1.) & 
                            (var_dict_cloud['precip_binary'] > -555.))
        #print('Total Cloud Sample Size: {}'.format(np.size(cloud_id)))
        samp_size = len(cloud_id)

        
        supercooled_cloud_id = np.where((var_dict_cloud['cbt'] < 0.) & 
                                        (var_dict_cloud['cloud_binary'] == 1.) & 
                                        (var_dict_cloud['precip_binary'] > -555.))
        
        warm_cloud_id = np.where((var_dict_cloud['cbt'] >= 0.) & 
                                 (var_dict_cloud['ctt'] >= 0.) & 
                                 (var_dict_cloud['cloud_binary'] == 1.) & 
                                 (var_dict_cloud['precip_binary'] > -555.))
        
        mixed_cloud_id = np.where((var_dict_cloud['cbt'] >= 0.) & 
                                  (var_dict_cloud['ctt'] < 0.) & 
                                  (var_dict_cloud['cloud_binary'] == 1.) & 
                                  (var_dict_cloud['precip_binary'] > -555.))
        
        supercooled_samp_size = len(supercooled_cloud_id)
        warm_samp_size = len(warm_cloud_id)
        mixed_samp_size = len(mixed_cloud_id)
        
        supercooled_frac = np.size(supercooled_cloud_id)/np.size(cloud_id)*100.
        warm_frac = np.size(warm_cloud_id)/np.size(cloud_id)*100.
        mixed_frac = np.size(mixed_cloud_id)/np.size(cloud_id)*100.
        #print('Sample Sizes:')
        #print('  Warm cloud % sample size:',np.size(warm_cloud_id)/np.size(cloud_id)*100.)
        #print('  Mixed cloud % sample size:',np.size(mixed_cloud_id)/np.size(cloud_id)*100.)
        #print('  Supercooled cloud % sample size:',np.size(supercooled_cloud_id)/np.size(cloud_id)*100.)
        
        precip_id = np.where(var_dict_cloud['precip_binary'] == 1.)
        supercooled_precip_id = np.where((var_dict_cloud['cbt'] < 0.) & (var_dict_cloud['precip_binary'] == 1))[0]
        warm_precip_id = np.where((var_dict_cloud['cbt'] >= 0.) & (var_dict_cloud['ctt'] >= 0.) & (var_dict_cloud['precip_binary'] == 1))[0]
        mixed_precip_id = np.where((var_dict_cloud['cbt'] >= 0.) & (var_dict_cloud['ctt'] < 0.) & (var_dict_cloud['precip_binary'] == 1))[0]
        
                             
        precip_occur_freq = np.size(precip_id)/np.size(cloud_id)*100.
        supercooled_precip_occur_freq = np.size(supercooled_precip_id)/np.size(supercooled_cloud_id)*100.
        warm_precip_occur_freq = np.size(warm_precip_id)/np.size(warm_cloud_id)*100.
        mixed_precip_occur_freq = np.size(mixed_precip_id)/np.size(mixed_cloud_id)*100.
                             
        #print('Precip occurrence frequencies:')
        #print('  All: {}'.format(precip_occur_freq))
        #print('  Supercooled: {}'.format(supercooled_precip_occur_freq))
        #print('  Warm: {}'.format(warm_precip_occur_freq))
        #print('  Mixed: {}'.format(mixed_precip_occur_freq))
         
        # SFC Precip
        sfc_cloud_id = np.where((var_dict_cloud['cloud_binary'] == 1.) &
                                (var_dict_cloud['sfc_precip_binary'] > -555.) & 
                                (var_dict_cloud['sfc_precip_updraft_binary'] != 1.) &
                                (var_dict_cloud['precip_binary'] > -555.))
        
        sfc_supercooled_cloud_id = np.where((var_dict_cloud['cbt'] < 0.) & 
                                            (var_dict_cloud['cloud_binary'] == 1.) & 
                                            (var_dict_cloud['sfc_precip_updraft_binary'] != 1) & 
                                            (var_dict_cloud['precip_binary'] > -555.) & 
                                            (var_dict_cloud['sfc_precip_binary'] > -555.))
        
        sfc_warm_cloud_id = np.where((var_dict_cloud['cbt'] >= 0.) & 
                                     (var_dict_cloud['ctt'] >= 0.) & 
                                     (var_dict_cloud['cloud_binary'] == 1.) & 
                                     (var_dict_cloud['sfc_precip_updraft_binary'] != 1.) & 
                                     (var_dict_cloud['precip_binary'] > -555.) & 
                                     (var_dict_cloud['sfc_precip_binary'] > -555.))
        
        sfc_mixed_cloud_id = np.where((var_dict_cloud['cbt'] >= 0.) &
                                      (var_dict_cloud['ctt'] < 0.) & 
                                      (var_dict_cloud['cloud_binary'] == 1.) & 
                                      (var_dict_cloud['sfc_precip_updraft_binary'] != 1.) & 
                                      (var_dict_cloud['precip_binary'] > -555.) & 
                                      (var_dict_cloud['sfc_precip_binary'] > -555.))
        
        sfc_supercooled_frac = np.size(sfc_supercooled_cloud_id)/np.size(sfc_cloud_id)*100.
        sfc_warm_frac = np.size(sfc_warm_cloud_id)/np.size(sfc_cloud_id)*100.
        sfc_mixed_frac = np.size(sfc_mixed_cloud_id)/np.size(sfc_cloud_id)*100.
        #print('Surface Sample Sizes:')
        #print('  Warm cloud % sample size:',np.size(sfc_warm_cloud_id)/np.size(sfc_cloud_id)*100.)
        #print('  Mixed cloud % sample size:',np.size(sfc_mixed_cloud_id)/np.size(sfc_cloud_id)*100.)
        #print('  Supercooled cloud % sample size:',np.size(sfc_supercooled_cloud_id)/np.size(sfc_cloud_id)*100.)    
        
        sfc_precip_id = np.where((var_dict_cloud['cloud_binary'] == 1.) &
                                             (var_dict_cloud['sfc_precip_updraft_binary'] == 0.) &
                                             (var_dict_cloud['precip_binary'] > -555.) &
                                             (var_dict_cloud['sfc_precip_updraft_binary'] != 1.) &
                                             (var_dict_cloud['sfc_precip_binary'] == 1.))
        
        sfc_supercooled_precip_id = np.where((var_dict_cloud['cbt'] < 0.) &
                                             (var_dict_cloud['ctt'] < 0.) &
                                             (var_dict_cloud['cloud_binary'] == 1.) &
                                             (var_dict_cloud['sfc_precip_updraft_binary'] != 1.) &
                                             (var_dict_cloud['precip_binary'] > -555.) &
                                             (var_dict_cloud['sfc_precip_binary'] == 1.))   
        
        sfc_warm_precip_id = np.where((var_dict_cloud['cbt'] >= 0.) &
                                             (var_dict_cloud['ctt'] >= 0.) &
                                             (var_dict_cloud['cloud_binary'] == 1.) &
                                             (var_dict_cloud['sfc_precip_updraft_binary'] != 1.) &
                                             (var_dict_cloud['precip_binary'] > -555.) &
                                             (var_dict_cloud['sfc_precip_binary'] == 1.))
        
        sfc_mixed_precip_id = np.where((var_dict_cloud['cbt'] >= 0.) &
                                             (var_dict_cloud['ctt'] < 0.) &
                                             (var_dict_cloud['cloud_binary'] == 1.) &
                                             (var_dict_cloud['sfc_precip_updraft_binary'] != 1.) &
                                             (var_dict_cloud['precip_binary'] > -555.) &
                                             (var_dict_cloud['sfc_precip_binary'] == 1.))
        
        
        sfc_precip_occur_freq = np.size(sfc_precip_id)/np.size(sfc_cloud_id)*100.
        sfc_supercooled_precip_occur_freq = np.size(sfc_supercooled_precip_id)/np.size(sfc_supercooled_cloud_id)*100.
        sfc_warm_precip_occur_freq = np.size(sfc_warm_precip_id)/np.size(sfc_warm_cloud_id)*100.
        sfc_mixed_precip_occur_freq = np.size(sfc_mixed_precip_id)/np.size(sfc_mixed_cloud_id)*100.
                         
            
        #print('SFC Precip occurrence frequencies:')
        #print('  All: {}'.format(sfc_precip_occur_freq))
        #print('  Supercooled: {}'.format(sfc_supercooled_precip_occur_freq))
        #print('  Warm: {}'.format(sfc_warm_precip_occur_freq))
        #print('  Mixed: {}'.format(sfc_mixed_precip_occur_freq)) 
        
        
        
        # PRECIP TO SFC
        precip_to_sfc_precip_id = np.where((var_dict_cloud['cloud_binary'] == 1.) &
                                             (var_dict_cloud['precip_binary'] == 1.) &
                                             (var_dict_cloud['precip_to_sfc_binary'] == 1.))
        
        precip_to_sfc_supercooled_precip_id = np.where((var_dict_cloud['cbt'] < 0.) &
                                             (var_dict_cloud['ctt'] < 0.) &
                                             (var_dict_cloud['cloud_binary'] == 1.) &
                                             (var_dict_cloud['precip_to_sfc_binary'] == 1.) &
                                             (var_dict_cloud['precip_binary'] ==1.))   
        
        precip_to_sfc_warm_precip_id = np.where((var_dict_cloud['cbt'] >= 0.) &
                                             (var_dict_cloud['ctt'] >= 0.) &
                                             (var_dict_cloud['cloud_binary'] == 1.) &
                                             (var_dict_cloud['precip_to_sfc_binary'] == 1.) &
                                             (var_dict_cloud['precip_binary'] == 1.))
        
        precip_to_sfc_mixed_precip_id = np.where((var_dict_cloud['cbt'] >= 0.) &
                                             (var_dict_cloud['ctt'] < 0.) &
                                             (var_dict_cloud['cloud_binary'] == 1.) &
                                             (var_dict_cloud['precip_to_sfc_binary'] == 1.) &
                                             (var_dict_cloud['precip_binary'] == 1.))
        
        
        precip_to_sfc_precip_occur_freq = np.size(precip_to_sfc_precip_id)/np.size(sfc_cloud_id)*100.
        precip_to_sfc_supercooled_precip_occur_freq = np.size(precip_to_sfc_supercooled_precip_id)/np.size(supercooled_cloud_id)*100.
        precip_to_sfc_warm_precip_occur_freq = np.size(precip_to_sfc_warm_precip_id)/np.size(warm_cloud_id)*100.
        precip_to_sfc_mixed_precip_occur_freq = np.size(precip_to_sfc_mixed_precip_id)/np.size(mixed_cloud_id)*100.
                         
            
        #print('PRECIP-TO-SFC occurrence frequencies:')
        #print('  All: {}'.format(precip_to_sfc_precip_occur_freq))
        #print('  Supercooled: {}'.format(precip_to_sfc_supercooled_precip_occur_freq))
        #print('  Warm: {}'.format(precip_to_sfc_warm_precip_occur_freq))
        #print('  Mixed: {}'.format(precip_to_sfc_mixed_precip_occur_freq)) 
        
        
        
        out_dict = {'precip_occur_freq':precip_occur_freq,\
                    'supercooled_precip_occur_freq':supercooled_precip_occur_freq,\
                    'warm_precip_occur_freq':warm_precip_occur_freq,\
                    'mixed_precip_occur_freq':mixed_precip_occur_freq,\
                    'supercooled_frac':supercooled_frac,\
                    'warm_frac':warm_frac,\
                    'mixed_frac':mixed_frac,\
                    'sfc_precip_occur_freq':sfc_precip_occur_freq,\
                    'sfc_supercooled_precip_occur_freq':sfc_supercooled_precip_occur_freq,\
                    'sfc_warm_precip_occur_freq':sfc_warm_precip_occur_freq,\
                    'sfc_mixed_precip_occur_freq':sfc_mixed_precip_occur_freq,\
                    'precip_to_sfc_precip_occur_freq':precip_to_sfc_precip_occur_freq,\
                    'precip_to_sfc_supercooled_precip_occur_freq':precip_to_sfc_supercooled_precip_occur_freq,\
                    'precip_to_sfc_warm_precip_occur_freq':precip_to_sfc_warm_precip_occur_freq,\
                    'precip_to_sfc_mixed_precip_occur_freq':precip_to_sfc_mixed_precip_occur_freq,\
                    'samp_size':samp_size,\
                    'supercooled_samp_size':samp_size,\
                    'warm_samp_size':samp_size,\
                    'mixed_samp_size':samp_size,\
                   }
        return out_dict          

In [ ]:
dmin_arr = np.arange(50,650,50)
ze_thresh_arr = np.array([-55,-50,-45,-40,-35,-30,-25,-20,-15,-10,-5,0,5,10,15])

keys = ['supercooled','warm','mixed','all']
precip_occur_freq_dict = {}
sfc_precip_occur_freq_dict = {}
precip_to_sfc_precip_occur_freq_dict = {}
samp_size_dict = {}
perc_dict = {}
for key in keys:
    precip_occur_freq_dict[key] = np.zeros([np.size(dmin_arr),np.size(ze_thresh_arr)])
    sfc_precip_occur_freq_dict[key] = np.zeros([np.size(dmin_arr),np.size(ze_thresh_arr)])
    precip_to_sfc_precip_occur_freq_dict[key] = np.zeros([np.size(dmin_arr),np.size(ze_thresh_arr)])
    samp_size_dict[key] = np.zeros([np.size(dmin_arr),np.size(ze_thresh_arr)])
    
    if key != 'all':
        perc_dict[key] = np.zeros([np.size(dmin_arr),np.size(ze_thresh_arr)])
        

In [ ]:
for ii in range(len(dmin_arr)):
    print(dmin_arr[ii])
    for jj in range(len(ze_thresh_arr)):
        #out_dict = calc_precip_occur_freq(dmin_arr[ii],ze_thresh_arr[jj])
        out_dict = calc_precip_occur_freq(dmin_arr[ii],ze_thresh_arr[jj],results_dict)
        #print(aaa)
        precip_occur_freq_dict['supercooled'][ii,jj] = out_dict['supercooled_precip_occur_freq']
        precip_occur_freq_dict['warm'][ii,jj] = out_dict['warm_precip_occur_freq'] 
        precip_occur_freq_dict['mixed'][ii,jj] = out_dict['mixed_precip_occur_freq'] 
        precip_occur_freq_dict['all'][ii,jj] = out_dict['precip_occur_freq']
        sfc_precip_occur_freq_dict['supercooled'][ii,jj] = out_dict['sfc_supercooled_precip_occur_freq'] 
        sfc_precip_occur_freq_dict['warm'][ii,jj] = out_dict['sfc_warm_precip_occur_freq'] 
        sfc_precip_occur_freq_dict['mixed'][ii,jj] = out_dict['sfc_mixed_precip_occur_freq'] 
        sfc_precip_occur_freq_dict['all'][ii,jj] = out_dict['sfc_precip_occur_freq']        
        precip_to_sfc_precip_occur_freq_dict['supercooled'][ii,jj] = out_dict['precip_to_sfc_supercooled_precip_occur_freq'] 
        precip_to_sfc_precip_occur_freq_dict['warm'][ii,jj] = out_dict['precip_to_sfc_warm_precip_occur_freq'] 
        precip_to_sfc_precip_occur_freq_dict['mixed'][ii,jj] = out_dict['precip_to_sfc_mixed_precip_occur_freq'] 
        precip_to_sfc_precip_occur_freq_dict['all'][ii,jj] = out_dict['precip_to_sfc_precip_occur_freq']  
        perc_dict['supercooled'][ii,jj] = out_dict['supercooled_frac'] 
        perc_dict['warm'][ii,jj] = out_dict['warm_frac'] 
        perc_dict['mixed'][ii,jj] = out_dict['mixed_frac']
        samp_size_dict['all'][ii,jj] = out_dict['samp_size']
        samp_size_dict['supercooled'][ii,jj] = out_dict['supercooled_samp_size']
        samp_size_dict['warm'][ii,jj] = out_dict['warm_samp_size']
        samp_size_dict['mixed'][ii,jj] = out_dict['mixed_samp_size']
        #client.run(trim_memory)
print('done')

In [ ]:
final_out_dict = {}
final_out_dict['precip_occur_freq'] = precip_occur_freq_dict
final_out_dict['sfc_precip_occur_freq'] = sfc_precip_occur_freq_dict
final_out_dict['precip_to_sfc_precip_occur_freq'] = precip_to_sfc_precip_occur_freq_dict
final_out_dict['perc'] = perc_dict
final_out_dict['samp_size'] = samp_size_dict

In [ ]:
save_path = '/mnt/raid/mwstanfo/'
#out_pkl_file = save_path+'micre_precip_occur_freq_varying_detections_final.p'
#out_pkl_file = save_path+'micre_precip_occur_freq_varying_detections_final_lowbias25.p'
out_pkl_file = save_path+'micre_precip_occur_freq_varying_detections_final_lowbias50.p'
pickle.dump(final_out_dict,open(out_pkl_file,"wb"))

In [ ]:
#final_out_dict['precip_occur_freq']['all']

In [ ]:
client.shutdown()